<a href="https://colab.research.google.com/github/jyasaswy/EE595-Project/blob/master/XNORnet_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [0]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import sys
import numpy
from torchvision import datasets, transforms
from torch.autograd import Variable
import warnings 
warnings.filterwarnings("ignore")

Binarize the Input Activations

In [0]:
class BinActive(torch.autograd.Function):
    '''
    Binarize the input activations and calculate the mean across channel dimension.
    '''
    @staticmethod
    def forward(self, input):
        self.save_for_backward(input)
        size = input.size()
        input = input.sign()
        return input
    @staticmethod
    def backward(self, grad_output):
        input, = self.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input.ge(1)] = 0
        grad_input[input.le(-1)] = 0
        return grad_input

Binary Convolution

In [0]:
class convSFQ(nn.Module):
    def __init__(self, input_channels, output_channels,
            kernel_size=-1, stride=-1, padding=-1, groups=1, dropout=0,
            Linear=False, previous_conv=False, size=0):
        super(convSFQ, self).__init__()
        self.input_channels = input_channels
        self.layer_type = 'convSFQ'
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dropout_ratio = dropout
        self.previous_conv = previous_conv
        print(input_channels, output_channels, kernel_size, stride, padding, groups)
        self.conv = nn.Conv2d(input_channels, output_channels,
                    kernel_size=kernel_size, stride=stride, padding=padding, groups=groups)
        
    def forward(self, x):        
        x = self.conv(x)
        return(x)

class BinConv2d(nn.Module): # change the name of BinConv2d
    def __init__(self, input_channels, output_channels,
            kernel_size=-1, stride=-1, padding=-1, groups=1, dropout=0,
            Linear=False, previous_conv=False, size=0):
        super(BinConv2d, self).__init__()
        self.input_channels = input_channels
        self.layer_type = 'BinConv2d'
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dropout_ratio = dropout
        self.previous_conv = previous_conv


    # (bn): BatchNorm2d(20, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
    # (conv): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
    # (relu): ReLU(inplace=True)


        if dropout!=0:
            self.dropout = nn.Dropout(dropout)
        self.Linear = Linear
        if not self.Linear:
            self.bn = nn.BatchNorm2d(input_channels, eps=1e-4, momentum=0.1, affine=True)
            #self.conv = nn.Conv2d(input_channels, output_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=groups)
            #print(input_channels, output_channels, kernel_size, stride, padding, groups)
            self.conv = convSFQ(input_channels, output_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=groups)
        else:
            if self.previous_conv:
                self.bn = nn.BatchNorm2d(int(input_channels/size), eps=1e-4, momentum=0.1, affine=True)
            else:
                self.bn = nn.BatchNorm1d(input_channels, eps=1e-4, momentum=0.1, affine=True)
            self.linear = nn.Linear(input_channels, output_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        x = self.bn(x)
        x = BinActive.apply(x)
        if self.dropout_ratio!=0:
            x = self.dropout(x)
        if not self.Linear:
            x = self.conv(x)
        else:
            if self.previous_conv:
                x = x.view(x.size(0), self.input_channels)
            x = self.linear(x)
        x = self.relu(x)
        return x

LeNet - 5 Model

In [0]:
class LeNet_5(nn.Module):
    def __init__(self):
        super(LeNet_5, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, kernel_size=5, stride=1)
        self.bn_conv1 = nn.BatchNorm2d(2, eps=1e-4, momentum=0.1, affine=False)
        self.relu_conv1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bin_conv2 = BinConv2d(2, 2, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bin_ip1 = BinConv2d(2*4*4, 500, Linear=True,
                previous_conv=True, size=4*4)
        self.ip2 = nn.Linear(500, 10)

        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                if hasattr(m.weight, 'data'):
                    m.weight.data.zero_().add_(1.0)
        return

    def forward(self, x):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                if hasattr(m.weight, 'data'):
                    m.weight.data.clamp_(min=0.01)
        #orch.Size([128, 1, 28, 28])
        x = self.conv1(x)           # torch.Size([128, 20, 24, 24])
        x = self.bn_conv1(x)        # torch.Size([128, 20, 24, 24])
        x = self.relu_conv1(x)      # torch.Size([128, 20, 24, 24])
        x = self.pool1(x)           # torch.Size([128, 20, 12, 12])
        x = self.bin_conv2(x)       # torch.Size([128, 50, 8, 8])
        x = self.pool2(x)           # torch.Size([128, 50, 4, 4])

        # x = x.view(x.size(0), 50*4*4)

        x = self.bin_ip1(x)         # torch.Size([128, 500])
        x = self.ip2(x)             # torch.Size([128, 10])
        return x

Binarize

In [0]:
class BinOp():
    def __init__(self, model):
        # count the number of Conv2d and Linear
        count_targets = 0
        for m in model.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                count_targets = count_targets + 1

        start_range = 1
        end_range = count_targets-2
        self.bin_range = numpy.linspace(start_range,
                end_range, end_range-start_range+1)\
                        .astype('int').tolist()
        self.num_of_params = len(self.bin_range)
        self.saved_params = []
        self.target_modules = []
        index = -1
        for m in model.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                index = index + 1
                if index in self.bin_range:
                    tmp = m.weight.data.clone()
                    self.saved_params.append(tmp)
                    self.target_modules.append(m.weight)
        return

    def binarization(self):
        self.meancenterConvParams()
        self.clampConvParams()
        self.save_params()
        self.binarizeConvParams()

    def meancenterConvParams(self):
        for index in range(self.num_of_params):
            s = self.target_modules[index].data.size()
            negMean = self.target_modules[index].data.mean(1, keepdim=True).\
                    mul(-1).expand_as(self.target_modules[index].data)
            self.target_modules[index].data = self.target_modules[index].data.add(negMean)
        

    def clampConvParams(self):
        for index in range(self.num_of_params):
            self.target_modules[index].data = \
                    self.target_modules[index].data.clamp(-1.0, 1.0)

    def save_params(self):
        for index in range(self.num_of_params):
            self.saved_params[index].copy_(self.target_modules[index].data)

    def binarizeConvParams(self):
        for index in range(self.num_of_params):
            n = self.target_modules[index].data[0].nelement()
            s = self.target_modules[index].data.size()
            if len(s) == 4:
                m = self.target_modules[index].data.norm(1, 3, keepdim=True)\
                        .sum(2, keepdim=True).sum(1, keepdim=True).div(n)
            elif len(s) == 2:
                m = self.target_modules[index].data.norm(1, 1, keepdim=True).div(n)
            self.target_modules[index].data = \
                    self.target_modules[index].data.sign().mul(m.expand(s))

    def restore(self):
        for index in range(self.num_of_params):
            self.target_modules[index].data.copy_(self.saved_params[index])

    def updateBinaryGradWeight(self):
        for index in range(self.num_of_params):
            weight = self.target_modules[index].data
            n = weight[0].nelement()
            s = weight.size()
            if len(s) == 4:
                m = weight.norm(1, 3, keepdim=True)\
                        .sum(2, keepdim=True).sum(1, keepdim=True).div(n).expand(s)
            elif len(s) == 2:
                m = weight.norm(1, 1, keepdim=True).div(n).expand(s)
            m[weight.lt(-1.0)] = 0 
            m[weight.gt(1.0)] = 0
            m = m.mul(self.target_modules[index].grad.data)
            m_add = weight.sign().mul(self.target_modules[index].grad.data)
            if len(s) == 4:
                m_add = m_add.sum(3, keepdim=True)\
                        .sum(2, keepdim=True).sum(1, keepdim=True).div(n).expand(s)
            elif len(s) == 2:
                m_add = m_add.sum(1, keepdim=True).div(n).expand(s)
            m_add = m_add.mul(weight.sign())
            self.target_modules[index].grad.data = m.add(m_add).mul(1.0-1.0/s[1]).mul(n)

Train, Test and Adjust Learning Rate Functions

In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()

        # process the weights including binarization
        bin_op.binarization()

        output = model(data)
        loss = criterion(output, target)
        loss.backward()

        # restore weights
        bin_op.restore()
        bin_op.updateBinaryGradWeight()

        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
    return

def test(evaluate=False):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0

    bin_op.binarization()
    j = 0
    for batch_idx, (data, target) in enumerate(test_loader):
    # for data, target in test_loader:
        j = j + 1
        if  j >0:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            test_loss += criterion(output, target).data.item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            # print('Test Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            #     epoch, batch_idx * len(data), len(train_loader.dataset),
            #     100. * batch_idx / len(train_loader), test_loss))

    bin_op.restore()
    
    acc = 100. * float(correct) / len(test_loader.dataset)
    if (acc > best_acc):
        best_acc = acc
        # if not evaluate:
            # save_state(model, best_acc)

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss * args.batch_size, correct, len(test_loader.dataset),
        100. * float(correct) / len(test_loader.dataset)))
    print('Best Accuracy: {:.2f}%\n'.format(best_acc))
    return

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 15 epochs"""
    lr = args.lr * (0.1 ** (epoch // args.lr_epochs))
    print('Learning rate:', lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr


Arguments

In [0]:
class BNN_ARGS:
  def __init__(self):
    self.batch_size = 128
    self.test_batch_size = 128
    self.epochs = 1
    self.lr_epochs = 15
    self.lr = 0.01
    self.momentum = 0.9
    self.weight_decay = 1e-5
    self.no_cuda = False
    self.seed = 1
    self.log_interval = 100
    self.arch = 'LeNet_5'
    self.pretrained = None
    self.evaluate = False
    self.cuda = not self.no_cuda and torch.cuda.is_available()

Main Code

In [93]:
args = BNN_ARGS()

print(args)
    
torch.manual_seed(args.seed)
if args.cuda:
  torch.cuda.manual_seed(args.seed)
    
    # load data
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
                ])),
            batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
            ])),
            batch_size=args.test_batch_size, shuffle=True, **kwargs)
    
    # generate the model
# test_loader = test_loader[:3000]
if args.arch == 'LeNet_5':
    model = LeNet_5()
else:
    print('ERROR: specified arch is not suppported')
    exit()

if not args.pretrained:
    best_acc = 0.0
else:
    pretrained_model = torch.load(args.pretrained)
    best_acc = pretrained_model['acc']
    model.load_state_dict(pretrained_model['state_dict'])

if args.cuda:
    model.cuda()
    
print(model)
param_dict = dict(model.named_parameters())
params = []
    
base_lr = 0.1
    
for key, value in param_dict.items():
    params += [{'params':[value], 'lr': args.lr,
        'weight_decay': args.weight_decay,
        'key':key}]
    
optimizer = optim.Adam(params, lr=args.lr,
        weight_decay=args.weight_decay)

criterion = nn.CrossEntropyLoss()

bin_op = BinOp(model)

if args.evaluate:
  test(evaluate=True)
  exit()

for epoch in range(1, args.epochs + 1):
  adjust_learning_rate(optimizer, epoch)
  train(epoch)
  test()

2 2 5 1 0 1
LeNet_5(
  (conv1): Conv2d(1, 2, kernel_size=(5, 5), stride=(1, 1))
  (bn_conv1): BatchNorm2d(2, eps=0.0001, momentum=0.1, affine=False, track_running_stats=True)
  (relu_conv1): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bin_conv2): BinConv2d(
    (bn): BatchNorm2d(2, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
    (conv): convSFQ(
      (conv): Conv2d(2, 2, kernel_size=(5, 5), stride=(1, 1))
    )
    (relu): ReLU(inplace=True)
  )
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bin_ip1): BinConv2d(
    (bn): BatchNorm2d(2, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
    (linear): Linear(in_features=32, out_features=500, bias=True)
    (relu): ReLU(inplace=True)
  )
  (ip2): Linear(in_features=500, out_features=10, bias=True)
)
Learning rate: 0.01
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.290885
Train Epoch: 1 [12800/60000 (21%)]	

In [0]:
class convSFQ_test(nn.Module):
    def __init__(self, input_channels, output_channels,
            kernel_size=-1, stride=-1, padding=-1, groups=1, dropout=0,
            Linear=False, previous_conv=False, size=0):
        super(convSFQ_test, self).__init__()
        self.input_channels = input_channels
        self.layer_type = 'convSFQ_test'
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dropout_ratio = dropout
        self.previous_conv = previous_conv
        print(input_channels, output_channels, kernel_size, stride, padding, groups)
        self.conv = nn.Conv2d(input_channels, output_channels,
                    kernel_size=kernel_size, stride=stride, padding=padding, groups=groups)
        
    def forward(self, x):
        # # print('Executing Custom Convolution')
        wt = self.conv.weight.data.numpy()
        w_sets = wt.shape[0]
        w_fets = wt.shape[1]
        w_rows = wt.shape[2]
        w_cols = wt.shape[3]

        zeroMat = torch.zeros(x.shape)
        input_BN = torch.where(x > 0, x, zeroMat)
        inputNP = input_BN.data.numpy()
        sets = inputNP.shape[0]
        fets = inputNP.shape[1]
        rows = inputNP.shape[2]
        cols = inputNP.shape[3]

        output = np.empty((sets, w_sets, rows-w_rows+1, cols-w_cols+1))

        # print(' sets = ', sets, ' fets = ', w_sets, ' Rows = ', rows-w_rows+1, ' Cols = ', cols-w_cols+1, ' F = ', w_fets, ' R = ', w_rows)
        # Total_Executions = sets*w_sets*(rows-w_rows+1)*(cols-w_cols+1)*w_fets*w_rows
        # ETC = (sets*w_sets*(rows-w_rows+1)*(cols-w_cols+1)*w_fets*w_rows)/3600
        # print('Estimated Time = ', ETC, ' to complete netlist executions = ', Total_Executions)

        for setNo in range(sets):
            for fetNo in range(w_sets):
                for rowNo in range(0,rows-w_rows+1,1):
                    for colNo in range(0,cols-w_cols+1,1):
                        temp = 0
                        for F in range(0,w_fets,1):
                            for R in range(w_rows):
                                for C in range(w_cols):
                                    temp = temp + int(not(numpy.logical_xor(bool(inputNP[setNo][F][R][C]), bool(wt[fetNo][F][R][C]))))
                        if temp > 4:
                            output[setNo,fetNo,rowNo,colNo] = 1
                        else:
                            output[setNo,fetNo,rowNo,colNo] = 0
        output = torch.tensor(output, dtype = torch.float)
        # output = self.conv(x)
        return(output)


def test_SFQ(evaluate=False):
    global best_acc
    model_test.eval()
    test_loss = 0
    correct = 0

    bin_op.binarization()
    j = 0
    for batch_idx, (data, target) in enumerate(test_loader):
    # for data, target in test_loader:
        j = j + 1
        if  j > 0:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data, volatile=True), Variable(target)
            output = model_test(data)
            test_loss += criterion(output, target).data.item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            # print('Test Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            #     epoch, batch_idx * len(data), len(train_loader.dataset),
            #     100. * batch_idx / len(train_loader), test_loss))
    
    acc = 100. * float(correct) / len(test_loader.dataset)
    if (acc > best_acc):
        best_acc = acc
        # if not evaluate:
            # save_state(model, best_acc)

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss * args.batch_size, correct, len(test_loader.dataset),
        100. * float(correct) / len(test_loader.dataset)))
    print('Best Accuracy: {:.2f}%\n'.format(best_acc))
    return


    """Sets the learning rate to the initial LR decayed by 10 every 15 epochs"""
    lr = args.lr * (0.1 ** (epoch // args.lr_epochs))
    print('Learning rate:', lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

In [110]:
import copy

weightData = copy.deepcopy(model.bin_conv2.conv.conv.weight.data)
oneMat = torch.ones(weightData.shape)
zeroMat = torch.zeros(weightData.shape)
weightData_N = torch.sigmoid(weightData)
weightData_BN = torch.where(weightData_N > 0.5, oneMat, zeroMat)
# print(weightData_BN)

model_test = copy.deepcopy(model)
bin_conv = convSFQ_test(2, 2, kernel_size=5, stride=1, padding = 0, groups=1)
model_test.bin_conv2.conv = bin_conv
model_test.bin_conv2.conv.conv.weight.data = copy.deepcopy(weightData_BN.data)
# print(model_test.bin_conv2.conv.conv.weight.data)

2 2 5 1 0 1


In [111]:
import numpy as np
test_SFQ()


Test set: Average loss: 6.2334, Accuracy: 1135/10000 (11.35%)
Best Accuracy: 81.70%

